O Perceptron, uma unidade fundamental em redes neurais, possui camadas: entrada (input) para dados, ocultas (hidden layers) e saída (output) para resultados.

Seu aprendizado ocorre em duas etapas: Propagação Direta (Forward Propagation) e Retropropagação (Backward Propagation):

Na Propagação Direta, cada neurônio recebe valores dos anteriores, ponderados pelos pesos e somados. Isso passa por função de ativação não linear. É vital para aprendizado, especialmente em tarefas complexas.

Este processo se repete para todos os neurônios nas camadas até a saída gerar resultados.

O resultado é avaliado por função de custo, minimizada para alto desempenho. Minimização envolve descida de gradiente, ajustando parâmetros.

O aprendizado está na Retropropagação, onde pesos e bias atualizam com derivadas em relação ao custo. Atualização segue:

Parâmetro atualizado = Parâmetro - derivada * learning rate

Learning rate, 0 a 1, guia taxa de movimento dos parâmetros rumo a mínimo. Escolher correto é vital, impacta eficiência da otimização na descida de gradiente.


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [8]:
data = pd.read_csv("creditcard.csv")
df = data.sample(frac=1)
print(df.shape)
df.head()

(284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
263221,160848.0,1.812322,-0.805928,-1.493664,0.267995,0.098902,0.230844,-0.192742,0.084907,0.831115,...,-0.005528,-0.258719,-0.052341,-1.462074,-0.166718,0.288492,-0.077725,-0.058050,139.95,0
8585,11574.0,-0.470738,0.371406,1.415019,0.417889,-0.767441,-0.253796,0.668798,-0.248302,1.495213,...,-0.120729,0.107825,0.028778,0.421878,-0.585630,1.099973,-0.022837,0.146680,138.22,0
108414,70886.0,-1.066721,0.408432,1.420904,0.890988,0.468988,-0.391520,1.101967,-0.406340,-0.746193,...,-0.378915,-0.797446,0.283403,0.081642,-0.038337,0.417571,-0.347904,-0.085699,69.95,0
217698,140994.0,2.113442,0.050843,-1.500615,0.268930,0.634220,-0.296820,0.223754,-0.262514,0.347780,...,-0.320032,-0.761359,0.256558,0.057941,-0.095130,0.022175,-0.054444,-0.054314,4.29,0
213970,139472.0,-0.687378,0.877389,0.725428,-0.507154,0.277876,0.427389,-0.099900,0.475242,0.241289,...,0.356397,1.119852,-0.253242,0.112419,-0.089989,-0.126535,-0.038203,0.110873,1.55,0


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.decomposition import PCA
import time
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.model_selection import train_test_split
import random


class MultiLayerPerceptron(BaseEstimator, ClassifierMixin): 
    def __init__(self, params=None):     
        if (params == None): 
            self.inputLayer = 30                       # Camada de Input
            self.hiddenLayer = 10                       # Camadas Ocultas
            self.OutputLayer = 2                       # Camadas de Output
            self.learningRate = 0.005                  # Taxa de aprendizado
            self.max_epochs = 600                      # Épocas
            self.BiasHiddenValue = -1                   # Bias das Camadas Ocultas
            self.BiasOutputValue = -1                  # Bias da Camada de Output
            self.activation = self.ativacao['sigmoid'] # Função de Ativação
            self.deriv = self.derivada['sigmoid']      # Derivada Função de Ativação
        else: 
            self.inputLayer = params['InputLayer']
            self.hiddenLayer = params['HiddenLayer']
            self.OutputLayer = params['OutputLayer']
            self.learningRate = params['LearningRate']
            self.max_epochs = params['Epocas']
            self.BiasHiddenValue = params['BiasHiddenValue']
            self.BiasOutputValue = params['BiasOutputValue']
            self.activation = self.ativacao[params['ActivationFunction']]
            self.deriv = self.derivada[params['ActivationFunction']]
        
        #Iniciando pesos e bias aleatoriamente
        self.WEIGHT_hidden = self.starting_weights(self.hiddenLayer, self.inputLayer)
        self.WEIGHT_output = self.starting_weights(self.OutputLayer, self.hiddenLayer)
        self.BIAS_hidden = np.array([self.BiasHiddenValue for i in range(self.hiddenLayer)])
        self.BIAS_output = np.array([self.BiasOutputValue for i in range(self.OutputLayer)])
        self.classes_number = 2
        
    pass
    
    def starting_weights(self, x, y): # pesos aleatorios
        return [[np.random.normal() for i in range(x)] for j in range(y)]
# Funções de ativação utilizadas
    ativacao = { 
         'sigmoid': (lambda x: 1/(1 + np.exp(-x))),
            'tanh': (lambda x: np.tanh(x)),
            'Relu': (lambda x: x*(x > 0)),
               }
    derivada = {
         'sigmoid': (lambda x: x*(1-x)),
            'tanh': (lambda x: 1-x**2),
            'Relu': (lambda x: 1 * (x>0))
               }
 
    def Backpropagation_Algorithm(self, x):
        DELTA_output = []
        # Erro: OutputLayer
        ERROR_output = self.output - self.OUTPUT_L2
        DELTA_output = ((-1)*(ERROR_output) * self.deriv(self.OUTPUT_L2))
        
        arrayStore = []
        # Atualizando pesos da OutputLayer e HiddenLayer
        for i in range(self.hiddenLayer):
            for j in range(self.OutputLayer):
                self.WEIGHT_output[i][j] -= (self.learningRate * (DELTA_output[j] * self.OUTPUT_L1[i]))
                self.BIAS_output[j] -= (self.learningRate * DELTA_output[j])
      
        
        delta_hidden = np.matmul(self.WEIGHT_output, DELTA_output)* self.deriv(self.OUTPUT_L1)
 
        # Atualizando pesos da HiddenLayer e InputLayer(x)
        for i in range(self.OutputLayer):
            for j in range(self.hiddenLayer):
                self.WEIGHT_hidden[i][j] -= (self.learningRate * (delta_hidden[j] * x[i]))
                self.BIAS_hidden[j] -= (self.learningRate * delta_hidden[j])
                

    def predict(self, X, y): 
       
        my_predictions = []
        for idx, inputs in enumerate(X): 

                # Forward
            self.OUTPUT_L1 = self.activation((np.dot(inputs, self.WEIGHT_hidden) + self.BIAS_hidden.T))
            self.OUTPUT_L2 = self.activation((np.dot(self.OUTPUT_L1, self.WEIGHT_output) + self.BIAS_output.T))

                #One-Hot
            if(max(self.OUTPUT_L2) == self.OUTPUT_L2[0]): 
                my_predictions.append(0)
            else:
                my_predictions.append(1)
   
        array_score = []
        print(my_predictions)
        mp = len(my_predictions)
        print(mp)
        for i in range(mp):
          if my_predictions[i] == 0: 
                array_score.append([i, 'Não fraudulento', my_predictions[i], y[i]])
          elif my_predictions[i] == 1:
                array_score.append([i, 'Fraudulento', my_predictions[i], y[i]])
                    
        dataframe = pd.DataFrame(array_score, columns=['_id', 'class', 'output', 'hoped_output'])

        return my_predictions, dataframe



    def fit(self, X, y):  
        count_epoch = 1
        total_error = 0
        n = len(X); 
        epoch_array = []
        error_array = []
        W0 = []
        W1 = []
        while(count_epoch <= self.max_epochs):
            for idx,inputs in enumerate(X): 
                self.output = np.zeros(self.classes_number)
                
                # Forward
                self.OUTPUT_L1 = self.activation((np.dot(inputs, self.WEIGHT_hidden) + self.BIAS_hidden.T))
                self.OUTPUT_L2 = self.activation((np.dot(self.OUTPUT_L1, self.WEIGHT_output) + self.BIAS_output.T))
           
                #One-Hot
                if(y[idx] == 0): 
                    self.output = np.array([1,0]) 
                elif(y[idx] == 1):
                    self.output = np.array([0,1]) 
                
                square_error = 0
                for i in range(self.OutputLayer):
                    erro = (self.output[i] - self.OUTPUT_L2[i])**2
                    square_error = (square_error + (0.05 * erro))
                    total_error = total_error + square_error
         
                # Backpropagation 
                self.Backpropagation_Algorithm(inputs)
                # Verificar o aprendizado do algoritmo
            total_error = (total_error / n)
            if((count_epoch % 10 == 0)or(count_epoch == 1)):
                print("Epoch ", count_epoch, "- Total error: ",total_error, f'\n Accuracy: {round((1-total_error)*100,2)}%')
                error_array.append(total_error)
                epoch_array.append(count_epoch)



                
            W0.append(self.WEIGHT_hidden)
            W1.append(self.WEIGHT_output)
             
                
            count_epoch += 1

        return self

In [11]:
colunas = df.columns
     

for i in colunas:
  df[i] = (df[i] - df[i].min()) / (df[i].max() - df[i].min())

In [ ]:
from sklearn.model_selection import train_test_split

X = df[colunas.drop('Class')]
Y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
     

X_test = np.array(X_test)
X_train = np.array(X_train)
y_train = np.array(y_train)
y_test = np.array(y_test)
     

dic = {'InputLayer':30, 'HiddenLayer':10, 'OutputLayer':2,
              'Epocas':30, 'LearningRate':0.05,'BiasHiddenValue':np.random.rand(), 
              'BiasOutputValue':np.random.rand(), 'ActivationFunction':'sigmoid'}

Perceptron = MultiLayerPerceptron(dic)
Perceptron.fit(X_train,y_train)


Epoch  1 - Total error:  0.00026339818764952697 
 Accuracy: 99.97%
Epoch  10 - Total error:  0.00025955296021166975 
 Accuracy: 99.97%
Epoch  20 - Total error:  0.00025954476061125904 
 Accuracy: 99.97%
Epoch  30 - Total error:  0.00025953687456325877 
 Accuracy: 99.97%
